In [8]:

# coding: utf-8

# In[3]:

import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Cropping2D
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.optimizers import Adam
import sklearn
from sklearn.model_selection import train_test_split
import theano
from theano import config
import os 
from keras.models import load_model
import matplotlib.image as mpimg
from random import shuffle
import theano.sandbox.cuda
#theano.sandbox.cuda.use("gpu0")
#os.environ['THEANO_FLAGS'] = "device=gpu" 
theano.config.floatX = 'float32'
#theano.config.device='gpu0'
learning_rate=0.00001


# In[15]:

def gamma(image):
    gamma = np.random.uniform(0.3, 1.5)
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

def generator(samples, batch_size=64):
    num_samples = len(samples)
    #print(generator)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                for i in range(3):
                    name = './data/IMG/'+batch_sample[i].split('/')[-1]
                    #name = './data/IMG/'+batch_sample[0].split('/')[-1]
                    #print(name)
                    correction=0.25          
                    center_image = mpimg.imread(name)#cv2.imread(name)
                    center_angle = float(batch_sample[3])
                    #center_image=crop_image(center_image)
                    if i==0:
                    #   angles.append(center_angle)
                        center_angle=center_angle
                    if i==1:
                        center_angle+=correction
                       #print('left')
                       #angles.append(center_angle+correction)
                    if i==2:
                        center_angle-=correction
                       #print('right')
                       #angles.append(center_angle-correction) 
                    #print(center_angle)
                    #plt.imshow(center_image)
                    #plt.show()
                    cropimg=center_image[60:140,:]
                    plt.imsave('crop_image.png',cropimg)
                    center_image=cv2.resize(center_image[60:140,:],(64,64))
                    plt.imsave('resize crop_image.png',center_image)
                    center_image = cv2.cvtColor(center_image, cv2.COLOR_BGR2RGB)
                    #center_image = cv2.GaussianBlur(center_image, (3,3), 0)
                    #recen=gamma(center_image)
                    plt.imsave('rgb crop_image.png',center_image)
                    #steername = str(center_angle) + 'steer' 
                    #saveimg=cv2.putText(center_image,steername,(0,0), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),1,cv2.LINE_AA)
                    #plt.suptitle("Angle: " + str(center_angle))
                    plt.figure()
                    if i==0:
                        plt.suptitle("center Angle: " + str(center_angle))
                        plt.imshow(center_image) 
                        plt.savefig('center_crop_image.png', format="png", bbox_inches='tight')
                    if i==1:
                        plt.suptitle("left Angle: " + str(center_angle))
                        plt.imshow(center_image)
                        plt.savefig('left_crop_image.png', format="png", bbox_inches='tight')
                    if i==2:
                        plt.suptitle("right Angle: " + str(center_angle))
                        plt.imshow(center_image)
                        plt.savefig('right_crop_image.png', format="png", bbox_inches='tight')
                    
                    images.append(center_image)
                    angles.append(center_angle)
                    #print(np.array(center_image).shape)
                    #images.append(recen)
                    #angles.append(center_angle)
                    images.append(cv2.flip(center_image,1))
                    sva=cv2.flip(center_image,1)
                    #steername = str(-1*center_angle) + 'steer' 
                    #saveimg=cv2.putText(sva,steername,(0,0), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),1,cv2.LINE_AA)
                    plt.figure()
                    plt.suptitle("Angle: " + str(-1*center_angle))
                    plt.imshow(sva)
                    plt.savefig('flip_crop_image.png', format="png", bbox_inches='tight')
                    angles.append(-1*center_angle)
                    #images.append(np.fliplr(recen))
                    #angles.append(-1*center_angle)
            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            #print(X_train.shape)
            #print(y_train.shape)
            yield sklearn.utils.shuffle(X_train, y_train)
            
            
def process_image(path,index):
    return img


# In[ ]:

print(theano.config.device)
lines=[]
with open('/home/sungmok/udacity/CarND-Behavioral-Cloning-P3-master/data/driving_log.csv') as csvfile:
    reader=csv.reader(csvfile)
    cnt=0
    for line in reader:
        if cnt>0:
            lines.append(line)
        cnt+=1



train_samples, validation_samples = train_test_split(lines, test_size=0.1)
#validation_samples, test_samples = train_test_split(validation, test_size=0.1)
#print(train_samples.shape)
#print(validation_samples.shape)
train_generator = generator(train_samples, batch_size=128)
validation_generator = generator(validation_samples, batch_size=128)
#test_generator = generator(test_samples, batch_size=64)
ch, row, col = 3, 64, 64  # Trimmed image format

model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1.0,        input_shape=(row, col,ch),output_shape=(row, col,ch)))
#model.add(Cropping2D(cropping=((40,20), (0,0)), input_shape=(row, col,ch)))
model.add(Convolution2D(24,3,3, border_mode='same', activation='relu', subsample=(2,2)))
model.add(MaxPooling2D(pool_size=(2,2),strides=(1,1)))       
model.add(Convolution2D(32, 3,3 ,border_mode='valid',subsample=(2,2)))
model.add(Activation('relu'))
model.add(Convolution2D(48,3,3,border_mode='valid',subsample=(1,1)))
model.add(Activation('relu'))
model.add(Convolution2D(64,3,3,border_mode='valid',subsample=(1,1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.summary()

model.compile(loss='mse', optimizer='adam')
history_object=model.fit_generator(train_generator, samples_per_epoch= len(train_samples),validation_data=validation_generator,nb_val_samples=len(validation_samples), nb_epoch=3)
#model.evaluate(test_generator, batch_size=3)
#model.save('model.h5')
### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()
#plt.imsave('cost.png',result)


cpu


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:155: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (3, 3), strides=(2, 2), activation="relu", padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:157: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), strides=(2, 2), padding="valid")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:159: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), strides=(1, 1), padding="valid")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:161: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="valid")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_8 (Lambda)            (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 32, 32, 24)        672       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 31, 31, 24)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 15, 15, 32)        6944      
_________________________________________________________________
activation_22 (Activation)   (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 13, 13, 48)        13872     
_________________________________________________________________
activation_23 (Activation)   (None, 13, 13, 48)        0         
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:172: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:172: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=804, steps_per_epoch=7232, validation_data=<generator..., epochs=3)`


Epoch 1/3


/usr/local/lib/python3.5/dist-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


KeyboardInterrupt: 